In [ ]:
import pandas as pd
import geopandas as gpd
import os
import ast
import warnings

warnings.filterwarnings("ignore")

# Base directory path
base_dir = r"C:\Users\natda\OneDrive - Northeastern University\Desktop\NatDave\Academics\PhD_NU\RESEARCH\Traffic_Stress\Boston"

# Construct file paths
roads_path = os.path.join(base_dir, "street_network.shp")
junctions_path = os.path.join(base_dir, "junctions.shp")
crossings_path = os.path.join(base_dir, "crossings.shp")
stop_signs_path = os.path.join(base_dir, "osm_stops.gdb")
signals_path = os.path.join(base_dir, "osm_signals.gdb")
unassociated_signals_path = os.path.join(base_dir, "unassociated_signals.shp")
unassociated_stop_signs_path = os.path.join(base_dir, "unassociated_stop_signs.shp")

# Load shapefiles
roads = gpd.read_file(roads_path)
junctions = gpd.read_file(junctions_path)
crossings = gpd.read_file(crossings_path)
signals = gpd.read_file(signals_path)
stop_signs = gpd.read_file(stop_signs_path)

# Create 'CONTROL' column in crossings shapefile
crossings['CONTROL'] = None

In [2]:
#########
# Signals
#########

# Reproject signals to match roads CRS
signals = signals.to_crs(roads.crs)

# Add 'ID' column to signals and assign sequential values
signals['ID'] = range(1, len(signals) + 1)

# Track unassociated signals
unassociated_signals = set(signals['ID'].tolist())

# Function to find the nearest junction for each signal
def find_nearest_junction(signal, junctions_gdf, threshold=40):
    """Find the closest junction within a given threshold (meters)."""
    distances = junctions_gdf.geometry.distance(signal.geometry)
    within_threshold = distances[distances <= threshold]
    
    if within_threshold.empty:
        return None  # No junctions within threshold
    
    # Return closest junction ID
    nearest_junction_idx = within_threshold.idxmin()
    return junctions_gdf.loc[nearest_junction_idx, 'JUNC_ID']

# Assign signals to junctions
junction_signal_map = {}  # Stores which junctions have signals

for _, signal in signals.iterrows():
    nearest_junction_id = find_nearest_junction(signal, junctions)

    if nearest_junction_id:
        junction_signal_map[nearest_junction_id] = True  # Mark this junction as having a signal
        unassociated_signals.discard(signal['ID'])

# Efficiently update crossings at junctions with signals
crossings.loc[crossings['JUNC_ID'].isin(junction_signal_map.keys()), 'CONTROL'] = "signal"

### Filtering signals near restricted roads
restricted_roads = roads[(roads['qExclude'].notna() & (roads['qExclude'] != 0)) |
                         (roads['qNoAccess'].notna() & (roads['qNoAccess'] != 0))]

# Create a buffer around restricted roads and find nearby signals
buffered_roads = restricted_roads.buffer(50)
buffered_roads_gdf = gpd.GeoDataFrame(geometry=buffered_roads, crs=restricted_roads.crs)

signals_near_restricted_roads = gpd.sjoin(signals, buffered_roads_gdf, how="inner", predicate="intersects")

# Remove these signals from unassociated list
signals_to_exclude = set(signals_near_restricted_roads['ID'])
unassociated_signals -= signals_to_exclude

# Save unassociated signals
unassociated_signals_gdf = signals[signals['ID'].isin(unassociated_signals)]
unassociated_signals_gdf.to_file(unassociated_signals_path, driver='ESRI Shapefile')

# Save updated crossings shapefile
crossings.to_file(crossings_path, driver='ESRI Shapefile')

print(f"Updated crossings.shp with signals. {len(unassociated_signals)} unassociated signals remain.")

Updated crossings.shp with signals. 59 unassociated signals remain.


In [3]:
crossings['CONTROL'].value_counts()

CONTROL
signal    6124
Name: count, dtype: int64

In [4]:
############
# Stop Signs
############

# Reproject stop signs to match roads CRS
stop_signs = stop_signs.to_crs(roads.crs)

# Add 'ID' column to stop signs and assign sequential values
stop_signs['ID'] = range(1, len(stop_signs) + 1)

# Create a set of assigned stop sign IDs
assigned_stop_signs = set()

# Find the road segments that at least partially fall within a threshold distance (meters) of a stop sign
def find_nearby_roads(stop_sign, roads_gdf, threshold=3.5):
    stop_buffer = stop_sign.geometry.buffer(threshold)  # Create a buffer around the stop sign
    nearby_roads = roads_gdf[roads_gdf.geometry.intersects(stop_buffer)]  # Find intersecting roads

    if nearby_roads.empty:
        return []  # No road segments within the threshold

    # Extract the segment portions within the buffer
    nearby_roads = nearby_roads.copy()
    nearby_roads["buffer_intersection"] = nearby_roads.geometry.intersection(stop_buffer)

    return nearby_roads[['unique_id', 'buffer_intersection']].values.tolist()  # Return list of (road_id, intersection portion)

# Dictionary mapping road segments to stop sign control, but tracking nearest junction
road_stop_control = {}

# Iterate through each stop sign and find all nearby road segments
for _, stop in stop_signs.iterrows():
    nearby_road_data = find_nearby_roads(stop, roads)

    if nearby_road_data:
        for road_id, intersect_geom in nearby_road_data:
            # Find the nearest junction to this segment portion
            junctions_nearby = junctions.copy()
            junctions_nearby["distance"] = junctions_nearby.geometry.distance(intersect_geom.centroid)
            nearest_junction = junctions_nearby.loc[junctions_nearby["distance"].idxmin(), "JUNC_ID"]

            # Store control only for the closest junction
            road_stop_control.setdefault(road_id, []).append((stop['ID'], nearest_junction))

        assigned_stop_signs.add(stop['ID'])

# Convert 'CRS_LEG' column to a list of road segment IDs
crossings['CRS_LEG'] = crossings['CRS_LEG'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Assign "stop" control only at crossings linked to the nearest junction
for idx, row in crossings.iterrows():
    crs_leg = row['CRS_LEG']  # List of road segment IDs

    if isinstance(crs_leg, list):
        for seg in crs_leg:
            if seg in road_stop_control:
                for stop_id, assigned_junction in road_stop_control[seg]:
                    if row['JUNC_ID'] == assigned_junction and row['CONTROL'] != "signal":
                        crossings.at[idx, 'CONTROL'] = "stop"
                        break  # Ensure we only assign once per crossing

# Save the updated crossings shapefile with stop sign assignments
crossings.to_file(crossings_path, driver='ESRI Shapefile')

# Identify unassociated stop signs
unassociated_stop_signs = stop_signs[~stop_signs['ID'].isin(assigned_stop_signs)]

# Save unassociated stop signs to a new shapefile
unassociated_stop_signs.to_file(unassociated_stop_signs_path, driver='ESRI Shapefile')

# Output the number of unassociated stop signs
print(f"There are {len(unassociated_stop_signs)} unassociated stop signs.")

There are 159 unassociated stop signs.


In [5]:
# =====================================================
# Identify Stop Signs Associated at 5m but Not at 3.5m
# =====================================================

# Find stop signs that are associated at 5m and at 3.5m
associated_stop_signs_5m = set()
associated_stop_signs_3_5m = set()

# Iterate through stop signs using both thresholds
for _, stop in stop_signs.iterrows():
    nearby_road_data_5m = find_nearby_roads(stop, roads, threshold=5)
    nearby_road_data_3_5m = find_nearby_roads(stop, roads, threshold=3.5)

    if nearby_road_data_5m:
        associated_stop_signs_5m.add(stop['ID'])
    if nearby_road_data_3_5m:
        associated_stop_signs_3_5m.add(stop['ID'])

# Stop signs that were associated at 5m but **are no longer associated at 3.5m**
newly_unassociated_stop_signs = stop_signs[
    stop_signs['ID'].isin(associated_stop_signs_5m - associated_stop_signs_3_5m)
]

# Define output path
newly_unassociated_stops_path = os.path.join(base_dir, "newly_unassociated_stop_signs_3_5m.shp")

# Save the stop signs that were associated at 5m but not at 3.5m
newly_unassociated_stop_signs.to_file(newly_unassociated_stops_path, driver="ESRI Shapefile")

# Output the number of stop signs in this category
print(f"There are {len(newly_unassociated_stop_signs)} stop signs that were associated at 5m but not at 3.5m.")

There are 39 stop signs that were associated at 5m but not at 3.5m.


In [6]:
crossings['CONTROL'].value_counts()

CONTROL
signal    6124
stop      2161
Name: count, dtype: int64

In [7]:
# ==========================================
# Assign Implied Stops (Case 1A: Four-Way)
# ==========================================

# Identify Junctions with Four Legs
four_leg_junctions = set(junctions.loc[junctions['NUM_LEGS'] == 4, 'JUNC_ID'])

# Identify One-Way Streets
one_way_segments = set(roads.loc[roads['StOperNEU'] == 1, 'unique_id'])

# Create a Dictionary to Group Crossings by JUNC_ID
junc_crossings = {j_id: crossings[crossings["JUNC_ID"] == j_id] for j_id in four_leg_junctions}

# Iterate Through Crossings to Detect Implied Stops
for j_id, group in junc_crossings.items():
    stop_ranks = {}  # Store CRS_RANK where a stop sign exists on a one-way street

    for idx, row in group.iterrows():
        crs_leg = row['CRS_LEG']
        if isinstance(crs_leg, list) and len(crs_leg) == 1:
            seg_id = crs_leg[0]
            if seg_id in one_way_segments and row['CONTROL'] == "stop":
                stop_ranks[row['CRS_RANK']] = seg_id  # Store the rank of the stop-controlled leg

    # Assign Implied Stops to Opposite Legs
    for stop_rank, seg_id in stop_ranks.items():
        opposite_rank = ((stop_rank - 1 + 2) % 4) + 1  # Get the opposite rank

        # Find the opposite leg crossing
        opposite_idx = group.index[group['CRS_RANK'] == opposite_rank]

        if not opposite_idx.empty:
            opp_row = crossings.loc[opposite_idx[0]]  # Fetch the first matching row

            opp_crs_leg = opp_row['CRS_LEG']
            if isinstance(opp_crs_leg, list) and len(opp_crs_leg) == 1:
                opp_seg_id = opp_crs_leg[0]

                if opp_seg_id in one_way_segments and (
                    pd.isna(opp_row['CONTROL']) or opp_row['CONTROL'] == ""
                ):  
                    crossings.at[opposite_idx[0], 'CONTROL'] = "implied_stop"


# ==========================================
# Assign Implied Stops (Case 1B: Three-Way)
# ==========================================

# Identify Junctions with Three Legs
three_leg_junctions = set(junctions.loc[junctions['NUM_LEGS'] == 3, 'JUNC_ID'])

# Create a Dictionary to Group Crossings by JUNC_ID
t_junc_crossings = {j_id: crossings[crossings["JUNC_ID"] == j_id] for j_id in three_leg_junctions}

# Iterate Through Three-Way Crossings to Assign Implied Stops
for j_id, group in t_junc_crossings.items():
    stop_legs = {}  # Store CRS_LEG where a stop sign exists on a one-way street

    for idx, row in group.iterrows():
        crs_leg = row['CRS_LEG']
        if isinstance(crs_leg, list) and len(crs_leg) == 1:
            seg_id = crs_leg[0]
            if seg_id in one_way_segments and row['CONTROL'] == "stop":
                stop_legs[seg_id] = row['CRS_STNM']  # Store the stop-controlled segment and its street name

    # Assign Implied Stops to the Other Part of the Crossbar
    for stop_seg, stop_street in stop_legs.items():
        # Find the opposite leg crossing (matching street name for FRM and TO but different for CRS)
        for idx, opp_row in group.iterrows():
            opp_crs_leg = opp_row['CRS_LEG']

            if (
                isinstance(opp_crs_leg, list) and len(opp_crs_leg) == 1 and  # Ensure single segment
                opp_crs_leg[0] in one_way_segments and  # Ensure the leg itself is one-way
                opp_row['CRS_STNM'] == stop_street and  # Match the street of the stop sign
                opp_row['CONTROL'] in [None, ""]  # Ensure no existing control
            ):
                crossings.at[idx, 'CONTROL'] = "implied_stop" 

# Save the updated crossings shapefile
crossings.to_file(crossings_path, driver='ESRI Shapefile')



# ==============================
# Assign Implied Stops (Case 2)
# ==============================

# Identify T-intersections (NUM_LEGS = 3)
t_junction_ids = set(junctions[junctions['NUM_LEGS'] == 3]['JUNC_ID'])

# Filter crossings that belong to T-intersections and are uncontrolled
t_crossings = crossings[
    (crossings['JUNC_ID'].isin(t_junction_ids)) & 
    (crossings['CONTROL'].isna() | (crossings['CONTROL'] == ""))
]

# Assign "implied_stop" to the stem of the T-intersection
for idx, row in t_crossings.iterrows():
    if row['FRM_STNM'] == row['TO_STNM'] and row['CRS_STNM'] != row['FRM_STNM']:
        crossings.at[idx, 'CONTROL'] = "implied_stop"

# Save the updated crossings shapefile
crossings.to_file(crossings_path, driver='ESRI Shapefile')

In [8]:
print(crossings['CONTROL'].value_counts(dropna=False))

CONTROL
None            26470
implied_stop     7198
signal           6124
stop             2161
Name: count, dtype: int64
